In [ ]:
provider = IBMQ.load_account()

In [9]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [10]:
#from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, IBMQ, transpile 

import qiskit
import math
import random
import numpy as np
import qiskit.quantum_info as qi
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.providers.aer import AerSimulator
from qiskit.tools.visualization import plot_histogram, plot_bloch_multivector

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import thermal_relaxation_error, amplitude_damping_error

In [ ]:
# Build noise circuit 

n_qubits = 1
noise_circ = QuantumCircuit(n_qubits)
noise_circ.h(0)
noise_circ.u(math.pi/3, 0, math.pi/6, 0)
noise_circ.x(0)
noise_circ.save_density_matrix(label = f'density')
noise_circ.measure_all()
print(noise_circ)

In [ ]:
# Building noise models 

## Bit-flip error - on measurement 
p_bitflip_1 = 0.1
p_bitflip_2 = 0.5
p_bitflip_3 = 0.9
error_bitflip_1 = pauli_error([('X',p_bitflip_1), ('I', 1 - p_bitflip_1)])
error_bitflip_2 = pauli_error([('X',p_bitflip_2), ('I', 1 - p_bitflip_2)])
error_bitflip_3 = pauli_error([('X',p_bitflip_3), ('I', 1 - p_bitflip_3)])
noise_bitflip_1 = NoiseModel()
noise_bitflip_2 = NoiseModel()
noise_bitflip_3 = NoiseModel()
noise_bitflip_1.add_all_qubit_quantum_error(error_bitflip_1, ['u'])
noise_bitflip_2.add_all_qubit_quantum_error(error_bitflip_2, ['u'])
noise_bitflip_3.add_all_qubit_quantum_error(error_bitflip_3, ['u'])

## Phase-flip error 
p_phaseflip_1 = 0.1
p_phaseflip_2 = 0.5
p_phaseflip_3 = 0.9
error_phaseflip_1 = pauli_error([('Z',p_phaseflip_1), ('I', 1 - p_phaseflip_1)])
error_phaseflip_2 = pauli_error([('Z',p_phaseflip_2), ('I', 1 - p_phaseflip_2)])
error_phaseflip_3 = pauli_error([('Z',p_phaseflip_3), ('I', 1 - p_phaseflip_3)])
noise_phaseflip_1 = NoiseModel()
noise_phaseflip_2 = NoiseModel()
noise_phaseflip_3 = NoiseModel()
noise_phaseflip_1.add_all_qubit_quantum_error(error_phaseflip_1, ['u'])
noise_phaseflip_2.add_all_qubit_quantum_error(error_phaseflip_2, ['u'])
noise_phaseflip_3.add_all_qubit_quantum_error(error_phaseflip_3, ['u'])

## Depolarizing error 
p_depolarizing_1 = 0.1
p_depolarizing_2 = 0.5
p_depolarizing_3 = 0.9
error_depolarizing_1 = depolarizing_error(p_depolarizing_1, 1)
error_depolarizing_2 = depolarizing_error(p_depolarizing_2, 1)
error_depolarizing_3 = depolarizing_error(p_depolarizing_3, 1)
noise_depolarizing_1 = NoiseModel()
noise_depolarizing_2 = NoiseModel()
noise_depolarizing_3 = NoiseModel()
noise_depolarizing_1.add_all_qubit_quantum_error(error_depolarizing_1, ['u'])
noise_depolarizing_2.add_all_qubit_quantum_error(error_depolarizing_2, ['u'])
noise_depolarizing_3.add_all_qubit_quantum_error(error_depolarizing_3, ['u'])

## Amplitude damping 
p_amp_damp_1 = 0.1
p_amp_damp_2 = 0.5
p_amp_damp_3 = 0.9
error_amp_damp_1 = amplitude_damping_error(p_amp_damp_1, 1)
error_amp_damp_2 = amplitude_damping_error(p_amp_damp_2, 1)
error_amp_damp_3 = amplitude_damping_error(p_amp_damp_3, 1)
noise_amp_damp_1 = NoiseModel()
noise_amp_damp_2 = NoiseModel()
noise_amp_damp_3 = NoiseModel()
noise_amp_damp_1.add_all_qubit_quantum_error(error_amp_damp_1, ['u'])
noise_amp_damp_2.add_all_qubit_quantum_error(error_amp_damp_2, ['u'])
noise_amp_damp_3.add_all_qubit_quantum_error(error_amp_damp_3, ['u'])

In [ ]:
# Define simulators 
sim_ideal = AerSimulator()
sim_noise = AerSimulator(noise_model = noise_bitflip_3)



In [ ]:
# Execute my ideal circuit
result_ideal = sim_ideal.run(noise_circ).result()


In [ ]:
result_ideal.get_counts()

In [ ]:
# Execute my noisy circuit
circ_tnoise = transpile(noise_circ, sim_noise)
result_bit_flip = sim_noise.run(circ_tnoise).result()

In [ ]:
result_bit_flip

In [ ]:
bit_flip_counts = result_bit_flip.get_counts()

In [ ]:
bit_flip_counts

In [ ]:
bit_flip_counts['0']/1024

In [ ]:
plot_histogram(bit_flip_counts)

In [ ]:
print(type(plot_histogram))

In [ ]:
result_bit_flip.data(0)

In [ ]:
plot_bloch_multivector(result_bit_flip.data(0)['density'])



In [12]:
def noisy_circuit_model(integer, p):
    """ function which computes the probablity """
    if integer == 0:
        error_bitflip_1 = pauli_error([('X',p), ('I', 1 - p)])
        noise_bitflip_1 = NoiseModel()
        noise_bitflip_1.add_all_qubit_quantum_error(error_bitflip_1, ['u'])
        sim_noise = AerSimulator(noise_model = noise_bitflip_1)
    elif integer == 1:
        error_phaseflip_1 = pauli_error([('Z',p), ('I', 1 - p)])
        noise_phaseflip_1 = NoiseModel()
        noise_phaseflip_1.add_all_qubit_quantum_error(error_phaseflip_1, ['u'])
        sim_noise = AerSimulator(noise_model = noise_phaseflip_1)
    elif integer == 2:
        error_depolarizing_1 = depolarizing_error(p, 1)
        noise_depolarizing_1 = NoiseModel()
        noise_depolarizing_1.add_all_qubit_quantum_error(error_depolarizing_1, ['u'])
        sim_noise = AerSimulator(noise_model = noise_depolarizing_1)
    elif integer == 3:
        error_amp_damp_1 = amplitude_damping_error(p, 1)
        noise_amp_damp_1 = NoiseModel()
        noise_amp_damp_1.add_all_qubit_quantum_error(error_amp_damp_1, ['u'])
        sim_noise = AerSimulator(noise_model = noise_amp_damp_1)
        
    n_qubits = 1
    noise_circ = QuantumCircuit(n_qubits)
    noise_circ.h(0)
    noise_circ.u(math.pi/3, 0, math.pi/6, 0)
    noise_circ.x(0)
    noise_circ.save_density_matrix(label = 'density')
    noise_circ.measure_all()
  
    circ_tnoise = transpile(noise_circ, sim_noise)
    result = sim_noise.run(circ_tnoise).result()
    counts = result.get_counts()
    
    prob_0 = counts['0']/1024
    prob_1 = counts['1']/1024
    
    result_prob = random.choices([0,1],(counts['0'],counts['1']))
    plot_bloch_multivector(result.data(0)['density']).savefig('./bloch_vector.png')
    plot_histogram(counts).savefig('./histogram.png')
    return result_prob[0]
        

In [13]:
noisy_circuit_model(0,0.8)

1